# Evaluating the chatbot

The ground truth data was manually created by the team and is stored in an Excel file. The file contains the following information:
1. Question
2. Correct answer
3. The ID's of the relevant chunks that should be returned by the retriever, per each embedding type.

In [1]:
from backend.evaluation.Evaluator import Evaluator
from backend.pipeline.DBHandler import DBHandler

import pandas as pd
import transformers
from retrying import retry

transformers.logging.set_verbosity_error()

In [2]:
@retry(stop_max_attempt_number=3, wait_fixed=60*1000)  # 3 attempts, 60 seconds between retries
def get_ground_truth_data(emb, gt_file="./FAQ.xlsx"):
	# Load ground truth data from a file
	data = pd.read_excel(gt_file)
	QA_list = []
	for i, row in data.iterrows():
		if not row.isnull().values.any():
			relevant_chunks_id = row[f"relevant_chunks_id_{emb}"].strip().split(",")
			QA_list.append((row["question"], row["answer"], relevant_chunks_id))
	return QA_list


In [3]:
def evaluate_chatbot(ground_truth_data, style, embedding_type, search_method, llm_name):
	db_handler = DBHandler(org_id=f's_maccabi_{embedding_type}', user_id='evaluator', search_method=search_method)
	evaluator = Evaluator(db_handler, style=style, llm_model_name=llm_name, embedding_model_name=embedding_type)
	results = evaluator.evaluate(ground_truth_data)
	results['style'] = style if style != '' else 'neutral'
	results['embedding_type'] = embedding_type
	results['search_method'] = search_method
	results['llm_name'] = llm_name
	return results

Evaluating the chatbot's answers with the ground truth data. Various configurations are tested:
1. 3 llms
2. 3 embedding types
3. 2 search methods
4. 5 styles

In [5]:
styles = ['', 'kids', 'elderly', 'emoji', 
		  'rhymes'
		  ]  # empty string means no style
embedding_types = ['emb1',  # models/text-embedding-004
				   'emb2',  # models/embedding-001
				   'emb3'  # from HW1: SentenceTransformer('all-MiniLM-L6-v2')
				   ]
llm_names = ['gemini-1.5-flash',
			 'meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo',
			 'mistralai/Mistral-7B-Instruct-v0.1'
			 ]
search_methods = ['approximate',
				  'exact'
				  ]

full_results = pd.DataFrame(
	columns=['style', 'embedding_type', 'search_method', 'llm_name', 'question', 'true_answer', 'chatbot_answer',
			 'cosine_similarity', 'correctness_score', 'faithfulness_score', 'retriever_scores'])

for style in styles:
	str_style = style if style != '' else 'neutral'
	for embedding_type in embedding_types:
		ground_truth_data = get_ground_truth_data(embedding_type, gt_file="./FAQ1.xlsx")
		for llm_name in llm_names:
			for search_method in search_methods:
				print(f"Style: {str_style}", end=" | ")
				print(f"embedding_type: {embedding_type}", end=" | ")
				print(f"llm_name: {llm_name}", end=" | ")
				print(f"search_method: {search_method} | ", end=" Status: ")
				try:
					results = evaluate_chatbot(ground_truth_data, style, embedding_type, search_method, llm_name)
					full_results = pd.concat([full_results, results], ignore_index=True)
					full_results.to_csv("full_resultsC.csv", index=False)  #re-save after each iteration to be safe :)
					print("Done")
				except Exception as e:
					print(f"Configuration failed")
					print(f"Error content: {e}", end="\n\n")
					continue

Style: neutral | embedding_type: emb1 | llm_name: gemini-1.5-flash | search_method: approximate |  Status: Done
Style: neutral | embedding_type: emb1 | llm_name: gemini-1.5-flash | search_method: exact |  Status: Done
Style: neutral | embedding_type: emb1 | llm_name: meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo | search_method: approximate |  Status: Done
Style: neutral | embedding_type: emb1 | llm_name: meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo | search_method: exact |  Status: Done
Style: neutral | embedding_type: emb1 | llm_name: mistralai/Mistral-7B-Instruct-v0.1 | search_method: approximate |  Status: Done
Style: neutral | embedding_type: emb1 | llm_name: mistralai/Mistral-7B-Instruct-v0.1 | search_method: exact |  Status: Done
Style: neutral | embedding_type: emb2 | llm_name: gemini-1.5-flash | search_method: approximate |  Status: Done
Style: neutral | embedding_type: emb2 | llm_name: gemini-1.5-flash | search_method: exact |  Status: Done
Style: neutral | embedding_type: emb2 

In [6]:
full_results.head(100)

,style,embedding_type,search_method,llm_name,question,true_answer,chatbot_answer,cosine_similarity,correctness_score,faithfulness_score,retriever_scores,precision,recall,f1
0,neutral,emb1,approximate,gemini-1.5-flash,Can I call my doctor on video?,Maccabi members are entitled to video medical ...,"Yes, you can schedule a video consultation wit...",0.743499,0.247833,0.692367,NaN,0.66667,1.0,0.8
1,neutral,emb1,exact,gemini-1.5-flash,Can I call my doctor on video?,Maccabi members are entitled to video medical ...,"Yes, you can schedule a video call with a doct...",0.740101,0.246700,0.695518,NaN,0.66667,1.0,0.8
2,neutral,emb1,approximate,meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo,Can I call my doctor on video?,Maccabi members are entitled to video medical ...,"Yes, you can call your doctor on video. You ar...",0.827942,0.275981,0.793553,NaN,0.66667,1.0,0.8
3,neutral,emb1,exact,meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo,Can I call my doctor on video?,Maccabi members are entitled to video medical ...,"Yes, you can call your doctor on video. You ar...",0.827942,0.275981,0.793553,NaN,0.66667,1.0,0.8
4,neutral,emb1,approximate,mistralai/Mistral-7B-Instruct-v0.1,Can I call my doctor on video?,Maccabi members are entitled to video medical ...,"Yes, it is possible to have a video call with...",0.843738,0.614579,0.852607,NaN,0.66667,1.0,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,rhymes,emb3,exact,gemini-1.5-flash,Can I call my doctor on video?,Maccabi members are entitled to video medical ...,"Oh, you wish to speak with your doctor, it see...",0.626613,0.208871,0.653262,NaN,0.66667,1.0,0.8
86,rhymes,emb3,approximate,meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo,Can I call my doctor on video?,Maccabi members are entitled to video medical ...,You want to call your doc with a smile so brig...,0.214080,0.071360,0.378715,NaN,0.66667,1.0,0.8
87,rhymes,emb3,exact,meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo,Can I call my doctor on video?,Maccabi members are entitled to video medical ...,You want to call your doc with a smile so brig...,0.214080,0.071360,0.378715,NaN,0.66667,1.0,0.8
88,rhymes,emb3,approximate,mistralai/Mistral-7B-Instruct-v0.1,Can I call my doctor on video?,Maccabi members are entitled to video medical ...,"Sure thing, let me check my notes.\n\nIt soun...",0.824307,0.608102,0.899693,NaN,0.66667,1.0,0.8
